# Package Install

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler

In [2]:
df=pd.read_csv('9_calories_data_2.csv')
df

,Condenser,Evaporator,RPM,Capacity
0,40,-35,1100,58.7
1,40,-30,1100,78.6
2,40,-25,1100,104.8
3,40,-20,1100,131.0
4,40,-15,1100,166.4
...,...,...,...,...
115,60,-30,3700,229.0
116,60,-25,3700,313.0
117,60,-20,3700,397.0
118,60,-15,3700,499.5


In [3]:
X=df.drop('Capacity',axis=1)
y=df['Capacity']

In [4]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

def performance(y_actual,y_predict):
    MAE=mean_absolute_error(y_actual,y_predict)
    print('MAE',MAE)
    MSE=mean_squared_error(y_actual,y_predict)
    print('MSE',MSE)
    R2=1-(np.sum((np.subtract(y_predict,y_actual))**2)/np.sum((np.subtract(y_actual,(np.sum(y_actual)/len(y_actual))))**2))
    print('R2: {}'.format(R2))
    

In [5]:
def feature(b_input,n_degree):
    polynomial_converter=PolynomialFeatures(degree=n_degree,include_bias=False)
    poly_features=polynomial_converter.fit_transform(b_input)
    return poly_features

In [6]:
def run_model(model,X_train,X_test,y_train,y_test):
    model.fit(X_train,y_train)
    predict=model.predict(X_test)
    performance(y_test,predict)

# Normalization

In [7]:
def score_normalize (n_data):
    mu=np.mean(n_data,axis=0)
    sigma=np.std(n_data,axis=0)
    X_norm=(n_data-mu)/sigma
    return (X_norm,mu,sigma)

# Data Scalling

In [29]:
a_cond=X['Condenser']
norm_cond,mu_cond,sigma_cond=score_normalize(a_cond)
a_eva=X['Evaporator']/100
norm_eva,mu_eva,sigma_eva=score_normalize(a_eva)
a_rpm=X['RPM']/1000
norm_rpm,mu_rpm,sigma_rpm=score_normalize(a_rpm)
X_normal=np.transpose([norm_cond, norm_eva,norm_rpm])

np.shape(X_normal)
# Ideal Data
X_input=X_normal
y_input=y
X_input=feature(X_input,1)
X_train,X_test,y_train,y_test=train_test_split(X_input,y_input,test_size=0.3,random_state=101)

scaler=StandardScaler()
scaler.fit(X_train)
scaler_X_train=scaler.transform(X_train)
scaler_X_test=scaler.transform(X_test)
#scaler_X_train=X_train
#scaler_X_test=X_test
# Test Value for performance cheeck
condenser=54.4
evaporator= -23.3
RPS=40
RPM=RPS*60
Real_value=226.8
Test_value=[[(condenser-mu_cond)/sigma_cond,(((evaporator/100)-mu_eva)/sigma_eva),((RPM/1000)-mu_rpm)/sigma_rpm]]
scaler_test=scaler.transform(feature(Test_value,1))
#scaler_test=Test_value
print(scaler_test)

[[ 0.59841978 -0.19140667  0.40860462]]


# Linear Model

In [32]:
from sklearn.linear_model import LinearRegression
linear_model=LinearRegression()

run_model(linear_model,scaler_X_train,scaler_X_test,y_train,y_test)

predict_test_linear=linear_model.predict(scaler_test)
predict_test_linear
print('Predicted Cooling Capacity: {}'.format(predict_test_linear))
deviation=np.abs(Real_value-predict_test_linear)/Real_value*100
print('Percentage of error:  {} %'.format(deviation))
print(linear_model.coef_)
print(scaler_test)

MAE 37.326745568827306
MSE 2521.8340894965295
R2: 0.8892990082787637
Predicted Cooling Capacity: [238.57537441]
Percentage of error:  [5.19196403] %
[-13.57951213  86.50867114 110.48742931]
[[ 0.59841978 -0.19140667  0.40860462]]


In [34]:
a=linear_model.coef_
b=scaler_test

In [36]:
np.sum(a*b)

20.461088700390206

In [37]:
predict_test_linear

array([238.57537441])

# Lasso Regression

In [10]:
from sklearn.linear_model import LassoCV
lasso_cv_model=LassoCV(eps=0.1,n_alphas=100,cv=5)
run_model(lasso_cv_model,scaler_X_train,scaler_X_test,y_train,y_test)
predict_test_lasso=lasso_cv_model.predict(scaler_test)
predict_test_lasso
deviation=np.abs(Real_value-predict_test_lasso)/Real_value*100
print('Predicted Cooling Capacity: {}'.format(predict_test_lasso))
print('Percentage of error:  {} %'.format(deviation))

MAE 13.60539819559857
MSE 370.1674946527395
R2: 0.9837507515138695
Predicted Cooling Capacity: [237.81689275]
Percentage of error:  [4.85753649] %


# Ridge Regression

In [11]:
#Maching Learning model
from sklearn.linear_model import Ridge
ridge_model=Ridge(alpha=10)
run_model(ridge_model,scaler_X_train,scaler_X_test,y_train,y_test)
predict_test_ridge=ridge_model.predict(scaler_test)
predict_test_ridge

deviation=np.abs(Real_value-predict_test_ridge)/Real_value*100
print('Predicted Cooling Capacity: {}'.format(predict_test_ridge))
print('Percentage of error:  {} %'.format(deviation))

MAE 14.156201355051179
MSE 314.330015780809
R2: 0.986201850224955
Predicted Cooling Capacity: [180.7775952]
Percentage of error:  [20.29206561] %


# Ridge CV

In [12]:
#Maching Learning model
from sklearn.linear_model import RidgeCV
ridge_cv_model=RidgeCV(alphas=(0.1,1.0,10.0),scoring='neg_mean_absolute_error')
run_model(ridge_cv_model,scaler_X_train,scaler_X_test,y_train,y_test)
predict_test_ridge_cv=ridge_cv_model.predict(scaler_test)
predict_test_ridge_cv
deviation=np.abs(Real_value-predict_test_ridge_cv)/Real_value*100
print('Predicted Cooling Capacity: {}'.format(predict_test_ridge_cv))
print('Percentage of error:  {} %'.format(deviation))

MAE 3.6178985724666677
MSE 35.891021559644315
R2: 0.9984244912474262
Predicted Cooling Capacity: [209.39267052]
Percentage of error:  [7.67518936] %


# Sunpport Vector Regression

In [13]:
from sklearn.svm import SVR,LinearSVR
SVR_model=SVR()
run_model(SVR_model,scaler_X_train,scaler_X_test,y_train,y_test)
predict_test_svr=SVR_model.predict(scaler_test)

deviation=np.abs(Real_value-predict_test_svr)/Real_value*100
print('Predicted Cooling Capacity: {}'.format(predict_test_svr))
print('Percentage of error:  {} %'.format(deviation))

MAE 104.08425901826405
MSE 22799.630472486057
R2: -0.0008357467662813978
Predicted Cooling Capacity: [176.24574745]
Percentage of error:  [22.29023481] %


# Grid Search CV

In [14]:
# Machine Learning Model
param_grid={'C':[0.001,0.01,0.1,0.5,1],
           'kernel':['linear','rbf','poly'],
           'gamma':['scale','auto'],
           'degree':[2,3,4,5],
           'epsilon':[0,0.01,0.1,0.5,1,2]}
from sklearn.model_selection import GridSearchCV
svr=SVR()
grid_model=GridSearchCV(svr,param_grid=param_grid)
run_model(grid_model,X_train,X_test,y_train,y_test)
predict_test_grid=grid_model.predict(scaler_test)
predict_test_grid

deviation=np.abs(Real_value-predict_test_grid)/Real_value*100
print('Predicted Cooling Capacity: {}'.format(predict_test_grid))
print('Percentage of error:  {} %'.format(deviation))

MAE 21.660783113311616
MSE 978.2963970548312
R2: 0.9570557072718042
Predicted Cooling Capacity: [154.65998474]
Percentage of error:  [31.80776687] %


# Decision Tree Regression

In [15]:
# Machine Learning Model
from sklearn.tree import DecisionTreeRegressor
DTree_model=DecisionTreeRegressor()
run_model(DTree_model,scaler_X_train,scaler_X_test,y_train,y_test)
predict_test_DTree=DTree_model.predict(scaler_test)
predict_test_DTree

deviation=np.abs(Real_value-predict_test_DTree)/Real_value*100
print('Predicted Cooling Capacity: {}'.format(predict_test_DTree))
print('Percentage of error:  {} %'.format(deviation))

MAE 9.416666666666673
MSE 246.47833333333378
R2: 0.9891803366242682
Predicted Cooling Capacity: [219.8]
Percentage of error:  [3.08641975] %


# KNN Regression

In [16]:
from sklearn.neighbors import KNeighborsRegressor
preds={}
k_values=[1,5,7,9]
for n in k_values:
    print('n_neighbors: {} \n'.format(n))
    knn_model=KNeighborsRegressor(n_neighbors=n)
    run_model(knn_model,scaler_X_train,scaler_X_test,y_train,y_test)
    predict_test_knn=knn_model.predict(scaler_test)
    print('Predicted Result: {}'.format(predict_test_knn))

    deviation=np.abs(Real_value-predict_test_knn)/Real_value*100
    print('Predicted Cooling Capacity: {}'.format(predict_test_knn))
    print('Percentage of error:  {} %'.format(deviation))
    print('\n')
    

n_neighbors: 1 

MAE 40.81944444444444
MSE 2486.6530555555555
R2: 0.8908433506933838
Predicted Result: [159.]
Predicted Cooling Capacity: [159.]
Percentage of error:  [29.89417989] %


n_neighbors: 5 

MAE 40.621111111111105
MSE 3032.663222222223
R2: 0.8668751335963008
Predicted Result: [171.78]
Predicted Cooling Capacity: [171.78]
Percentage of error:  [24.25925926] %


n_neighbors: 7 

MAE 44.79087301587302
MSE 3878.7917290249443
R2: 0.829732616879283
Predicted Result: [163.37142857]
Predicted Cooling Capacity: [163.37142857]
Percentage of error:  [27.96674225] %


n_neighbors: 9 

MAE 46.811728395061735
MSE 4910.283347050755
R2: 0.7844532126777177
Predicted Result: [170.16666667]
Predicted Cooling Capacity: [170.16666667]
Percentage of error:  [24.97060553] %




# Random Forest

In [17]:
# Machine learning 
from sklearn.ensemble import RandomForestRegressor
trees=[10,50,100]
for n in trees:
    print('Trees: ',n)
    RTree_model=RandomForestRegressor(n_estimators=n)
    run_model(RTree_model,scaler_X_train,scaler_X_test,y_train,y_test)
    predict_test_RTree=RTree_model.predict(scaler_test)
    print('Predicted Result: {}'.format(predict_test_RTree))
    deviation=np.abs(Real_value-predict_test_RTree)/Real_value*100
    print('Predicted Cooling Capacity: {}'.format(predict_test_RTree))
    print('Percentage of error:  {} %'.format(deviation))
    print('\n')
    

Trees:  10
MAE 13.749722222222227
MSE 436.6843416666672
R2: 0.9808308604071229
Predicted Result: [188.33]
Predicted Cooling Capacity: [188.33]
Percentage of error:  [16.96208113] %


Trees:  50
MAE 12.073999999999977
MSE 234.10162222222112
R2: 0.9897236373116346
Predicted Result: [188.022]
Predicted Cooling Capacity: [188.022]
Percentage of error:  [17.0978836] %


Trees:  100
MAE 12.133166666666645
MSE 278.2353945000005
R2: 0.9877862963977743
Predicted Result: [180.595]
Predicted Cooling Capacity: [180.595]
Percentage of error:  [20.37257496] %




# Gradient Boosting

In [18]:
#Machine Leanring Model
from sklearn.ensemble import GradientBoostingRegressor
Gradient_model=GradientBoostingRegressor()
run_model(Gradient_model,scaler_X_train,scaler_X_test,y_train,y_test)
predict_test_gradient=Gradient_model.predict(scaler_test)


deviation=np.abs(Real_value-predict_test_gradient)/Real_value*100
print('Predicted Cooling Capacity: {}'.format(predict_test_gradient))
print('Percentage of error:  {} %'.format(deviation))


MAE 5.888726304169036
MSE 72.87053700390265
R2: 0.9968012008612345
Predicted Cooling Capacity: [199.19870271]
Percentage of error:  [12.16988417] %


# AdaBoost

In [19]:
#Machine Leanring Model
from sklearn.ensemble import AdaBoostRegressor
ada_model=AdaBoostRegressor()
run_model(ada_model,scaler_X_train,scaler_X_test,y_train,y_test)
predict_test_adaboost=ada_model.predict(scaler_test)

deviation=np.abs(Real_value-predict_test_adaboost)/Real_value*100
print('Predicted Cooling Capacity: {}'.format(predict_test_adaboost))
print('Percentage of error:  {} %'.format(deviation))


MAE 20.821050102505552
MSE 614.3152280597938
R2: 0.9730333945207104
Predicted Cooling Capacity: [188.9125]
Percentage of error:  [16.70524691] %


# Test Result

In [20]:
print('Gradient Boost: %0.4f'%predict_test_gradient[0])
print('AdaBoost: %0.4f'%predict_test_adaboost[0])


Gradient Boost: 199.1987
AdaBoost: 188.9125


In [21]:
model_result={'Linear Model: {}':predict_test_linear[[0]],
             'Lasso Regression':predict_test_lasso[[0]]}

In [22]:
model_result

{'Linear Model: {}': array([219.17272621]),
 'Lasso Regression': array([237.81689275])}